In [39]:
from data_engineering import *
from porfolio_and_backtesting import *

In [40]:
end = dt(2022, 10, 4)
start = dt(2015, 10, 18)

In [41]:
df = getData(start, end, [], True)
tickers = ["AAPL"]*len(df.columns)

Data was loaded sucessfully!


In [42]:
w = np.array([[1 / (len(df.columns))]*len(df.columns),])

In [43]:
w = w.reshape(-1)

In [44]:
sp500 = Portfolio(df, tickers, w, start, end)